In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tkinter import Tk, Label, Entry, Button

In [2]:
# Load the dataset
dataset_path = "House_Price_dataset.csv"  # Update with the correct dataset path
data = pd.read_csv(dataset_path)

In [6]:
print(data.columns)

Index(['property_id', 'location_id', 'page_url', 'property_type', 'price',
       'location', 'city', 'province_name', 'latitude', 'longitude', 'baths',
       'area', 'purpose', 'bedrooms', 'date_added', 'agency', 'agent',
       'Area Type', 'Area Size', 'Area Category'],
      dtype='object')


In [18]:
# Identify non-numeric columns
non_numeric_columns = data.select_dtypes(include=['object']).columns.tolist()

In [19]:
# Exclude non-numeric columns from feature_columns
feature_columns = [col for col in data.columns if col != target_column and col not in non_numeric_columns]

In [13]:
# Drop rows with non-numeric values in the 'feature_columns'
data = data.dropna(subset=feature_columns)

In [14]:
# # Ensure correct column names
# target_column = 'price'  # Update with the correct column name of your target variable
# # feature_columns = [col for col in data.columns if col != target_column]

In [20]:
# Preprocessing
X = data[feature_columns].values
y = data[target_column].values

In [21]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [26]:
# Build the Neural Network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

In [27]:
# Compile the model
model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error', metrics=['mae'])

In [29]:
# Define callbacks
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test), callbacks=[checkpoint])


Epoch 1/10


3102/3110 [============================>.] - ETA: 0s - loss: 1750167497736192.0000 - mae: 19014602.0000
Epoch 1: val_loss improved from inf to 1801252644061184.00000, saving model to best_model.h5


G:\Anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3110/3110 [==============================] - 19s 5ms/step - loss: 1747908613373952.0000 - mae: 19006980.0000 - val_loss: 1801252644061184.0000 - val_mae: 19125446.0000
Epoch 2/10
3103/3110 [============================>.] - ETA: 0s - loss: 1639086087471104.0000 - mae: 18185920.0000
Epoch 2: val_loss improved from 1801252644061184.00000 to 1652903131480064.00000, saving model to best_model.h5
3110/3110 [==============================] - 17s 5ms/step - loss: 1655178256187392.0000 - mae: 18199826.0000 - val_loss: 1652903131480064.0000 - val_mae: 17849740.0000
Epoch 3/10
3102/3110 [============================>.] - ETA: 0s - loss: 1493320803024896.0000 - mae: 16709113.0000
Epoch 3: val_loss improved from 1652903131480064.00000 to 1474536226684928.00000, saving model to best_model.h5
3110/3110 [==============================] - 15s 5ms/step - loss: 1491890578915328.0000 - mae: 16703505.0000 - val_loss: 1474536226684928.0000 - val_mae: 16251183.0000
Epoch 4/10
3108/3110 [====================

In [30]:
# Load the best model
best_model = tf.keras.models.load_model("best_model.h5")

In [31]:
# Create GUI
def predict_price():
    try:
        features = [float(entry.get()) for entry in entries]
        features = np.array(features).reshape(1, -1)
        features = sc.transform(features)
        predicted_price = best_model.predict(features)
        result_label.config(text=f"Predicted Price: {predicted_price[0][0]}")
    except Exception as e:
        result_label.config(text=f"Error: {str(e)}")

# Initialize GUI
root = Tk()
root.title("House Price Predictor")

# Create input fields
entries = []
labels = feature_columns  # Use feature columns as labels

for i, label_text in enumerate(labels):
    label = Label(root, text=label_text)
    label.grid(row=i, column=0, padx=10, pady=10)
    entry = Entry(root)
    entry.grid(row=i, column=1, padx=10, pady=10)
    entries.append(entry)

# Create button to predict price
predict_button = Button(root, text="Predict Price", command=predict_price)
predict_button.grid(row=len(labels), columnspan=2, padx=10, pady=10)

# Label to display result
result_label = Label(root, text="")
result_label.grid(row=len(labels) + 1, columnspan=2, padx=10, pady=10)

root.mainloop()

1/1 [==============================] - 0s 310ms/step
